THE SCRIPT CANT BE RUN ON ITS OWN. 

This script assumes that embeddings and clustring has been performed already, and relies on resulting files! 

This script saves the files which is used by the analysis jupyter notebook 

In [2]:
import os
import polars as pl
import pandas as pd
import torch
import numpy as np
from sklearn.cluster import KMeans

In [3]:
# this cell finds all the specific STs for homolog groups.

# based on where the tipping point of the clusters is => core to shell

path = "/data/nilar/fastas_for_calibration/cluster_faiss_files"

homolog_group2ST = {}

if not os.path.isdir(path):
    print(f"Error: Directory not found at '{path}'")
else:
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if file.endswith("all_embeddings_fuzz_clustering_v2.csv"):
            if os.path.isfile(file_path):
                try:

                    df = pl.read_csv(file_path, separator=',', has_header=True)

                    for col_name in df.columns:
                        if col_name.endswith("category"):
                            

                            if not df[col_name].eq("core").any():
                                
                                split_name = col_name.split('_')
                                
                                if len(split_name) > 1:
                                    file_key = os.path.splitext(file)[0]
                                    file_key = file_key.replace("_all_embeddings_fuzz_clustering_v2", "").replace(" ", "_")
                                    homolog_group2ST[file_key] = float(split_name[1])
                                    
                                    break
                                    
                except Exception as e:
                    print(f"Could not process file {file}: {e}")

print(homolog_group2ST)

{'2-C-methyl-D-erythritol_2,4-cyclodiphosphate_synthase': 0.8997, '3-methyl-2-oxobutanoate_hydroxymethyltransferase': 0.9646, '4-hydroxy-3-methylbut-2-en-1-yl_diphosphate_synthase': 0.9381, '4-hydroxy-3-methylbut-2-enyl_diphosphate_reductase': 0.9174, 'ATP_cob(I)alamin_adenosyltransferase': 0.9351, 'ATP--corrinoid_adenosyltransferase': 0.9469, 'RNA_methyltransferase': 0.9292, 'Translation_initiation_factor': 0.9499, 'tRNA_(5-methylaminomethyl-2-thiouridylate)-methyltransferase': 0.9351, 'ligase': 0.9528, 'methyltransferase': 0.9558, '4-diphosphocytidyl-2-C-methyl-D-erythritol_kinase': 0.9351, 'DNA-directed_RNA_polymerase_subunit_beta': 0.9823, 'Ribosomal_RNA_small_subunit_methyltransferase_I': 0.9499, '2-oxoglutarate_dehydrogenase_E1': 0.9528, 'biotin_synthase': 0.9351, 'succinyltransferase': 0.9499, '2-amino-4-hydroxy-6-hydroxymethyldihydropteridine': 0.9233, 'acetolactate_synthase_small_subunit': 0.9617, 'phosphoglycerate_mutase_2_3-bisphosphoglycerate-independent': 0.9558, '2-succin

In [4]:
# this cell saves all embs into a single df


data_directory = '/data/nilar/fastas_for_calibration/busco_fastas_indiva/'
all_protein_data = []

homolog_groups = list(homolog_group2ST.keys())

for group_name in homolog_groups:
    group_path = os.path.join(data_directory, group_name)
    
    if os.path.isdir(group_path):
        pt_file = next((f for f in os.listdir(group_path) if f.endswith('.pt')), None)
        
        if pt_file:
            file_path = os.path.join(group_path, pt_file)
            try:
                embeddings_dict = torch.load(file_path)
                optimal_st = homolog_group2ST[group_name]
                
                for protein_id, embedding in embeddings_dict.items():
                    all_protein_data.append({
                        'protein_id': protein_id,
                        'homolog_group': group_name,
                        'st': optimal_st,
                        'embedding': embedding.numpy() 
                    })
            except Exception as e:
                print(f"Could not process file {file_path}: {e}")

df = pd.DataFrame(all_protein_data)



In [10]:
# this cell performs K-Means clustering to find representative embeddings

K_REPRESENTATIVES = 20 

representative_embeddings = []
print(f"Starting K-Means sampling to select {K_REPRESENTATIVES} representatives per group...")

for group_name, group_df in df.groupby('homolog_group'):
    
    embeddings_matrix = np.vstack(group_df['embedding'].values)
    
    n_samples = embeddings_matrix.shape[0]
    if n_samples == 0:
        continue
    
    k = min(K_REPRESENTATIVES, n_samples)

    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(embeddings_matrix)
    
    cluster_centers = kmeans.cluster_centers_
    
    for center_embedding in cluster_centers:
        representative_embeddings.append({
            'homolog_group': group_name,
            'st': group_df['st'].iloc[0], 
            'embedding': center_embedding 
        })

df_representatives = pd.DataFrame(representative_embeddings)

df_representatives_to_save = df_representatives.copy()
df_representatives_to_save['embedding'] = df_representatives_to_save['embedding'].apply(
    lambda arr: ' '.join(map(str, arr))
)
df_representatives_to_save.to_csv("/data/nilar/pan_genome/gaussian_process_regression_on_STs/bayesian_similarity/kMeans_embeddings.csv", index=False)

num_groups = len(df_representatives['homolog_group'].unique())

print("\nSampling complete.")
print(f"Created a new dataset with {len(df_representatives)} total representative embeddings.")
print(f"({num_groups} groups x {K_REPRESENTATIVES} representatives/group approx.)")

Starting K-Means sampling to select 20 representatives per group...

Sampling complete.
Created a new dataset with 640 total representative embeddings.
(32 groups x 20 representatives/group approx.)


In [9]:
# this cell performs random sampling to find representative embeddings

SAMPLES_PER_GROUP = 20

all_sampled_dfs = []
print(f"Starting random sampling to select {SAMPLES_PER_GROUP} embeddings per group...")

for group_name, group_df in df.groupby('homolog_group'):
    
    n_available = len(group_df)

    n_to_sample = min(SAMPLES_PER_GROUP, n_available)
    
    if n_to_sample > 0:
        sampled_data = group_df.sample(n=n_to_sample, random_state=42)
        all_sampled_dfs.append(sampled_data)

df_representatives = pd.concat(all_sampled_dfs, ignore_index=True)

df_representatives_to_save = df_representatives.copy()
df_representatives_to_save['embedding'] = df_representatives_to_save['embedding'].apply(
    lambda arr: ' '.join(map(str, arr))
)
df_representatives_to_save.to_csv("/data/nilar/pan_genome/gaussian_process_regression_on_STs/bayesian_similarity/kMeans_embeddings.csv", index=False)

num_groups = len(df['homolog_group'].unique())
print("\nSampling complete.")
print(f"Created a new dataset with {len(df_representatives)} total sampled embeddings.")
print(f"({num_groups} groups x up to {SAMPLES_PER_GROUP} samples/group)")

Starting random sampling to select 20 embeddings per group...

Sampling complete.
Created a new dataset with 640 total sampled embeddings.
(32 groups x up to 20 samples/group)


In [ ]:
# this cell performs a mean computaion of all the embs in each group

mean_embeddings_list = []
for group_name, group_df in df.groupby('homolog_group'):
    mean_embedding = np.mean(np.vstack(group_df['embedding'].values), axis=0)
    mean_embeddings_list.append({
        'homolog_group': group_name,
        'mean_embedding': mean_embedding,
        'st': group_df['st'].iloc[0] 
    })

mean_embeddings_df = pd.DataFrame(mean_embeddings_list)

mean_embeddings_df_to_save = mean_embeddings_df.copy()
mean_embeddings_df_to_save['mean_embedding'] = mean_embeddings_df_to_save['mean_embedding'].apply(
    lambda arr: ' '.join(map(str, arr))
)
mean_embeddings_df_to_save.to_csv("/data/nilar/pan_genome/gaussian_process_regression_on_STs/bayesian_similarity/mean_embeddings.csv", index=False)
print("Mean embeddings saved correctly to CSV.")

Mean embeddings saved correctly to CSV.
